## Configure environment

In [ ]:
import copernicus_marine_client as cmc #may return a warning at first (known CMC issue)
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import regionmask

## Start defining data characteristics
We want to access data:
* from [GLORYS12 (GLOBAL_MULTIYEAR_PHY_001_030)](https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/description) RAN system
* from [GLO12 (GLOBAL_ANALYSISFORECAST_PHY_001_024)](https://data.marine.copernicus.eu/product/GLOBAL_ANALYSISFORECAST_PHY_001_024/description) NRT Analysis and Forecasting system
* on SST (thetao, surface level)
* over a climatological period (reference = 1993-2016) and on latest (or previous) forecasting dates
* computed over one of the Icelandic ZEE subregions shapefiles (delivered with notebook)

In [ ]:
#Get GLORYS12 Reanalysis characteristics
product_id=['GLOBAL_MULTIYEAR_PHY_001_030'] #Unused unless you need to access the dataset's metadata
dataset_id=['cmems_mod_glo_phy_my_0.083_P1D-m']

# period=["2014-01-01", "2016-12-31"] #SHORT PERIOD FOR TESTING CLIMATOLOGY COMPUTATION
period=["1993-01-01", "2016-12-31"]
variables=["thetao"]

#Get GLO12V4 Forecasting characteristics
product_id=['GLOBAL_ANALYSISFORECAST_PHY_001_024'] #Unused unless you need to access the dataset's metadata
dataset_id_fcst=['cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m']
period_fcst=[datetime.now().strftime('%Y-%m-%d'),(datetime.now()+timedelta(days=9)).strftime('%Y-%m-%d')] #Latest forecast
# period_fcst=["2023-06-11", "2023-06-20"] #Hindcast only

#Get Shapefiles:
shp_dir= "./data/SHAPEFILES/ICELAND_SUBREGIONS/"
region_ID = 3 #(1= easternmost region, then rotating clockwise up to 9)

## Start opening data
* read shapefiles
* open ZARR geo-chunked datasets for RAN (faster on timeseries)
* open ZARR time-chunked datasets for forecasts (faster over space)

***WARNING**: CMC credentials are read from environement variables (through Vault)*

### Load Shapefile data

Manually processed shapefiles. Subregion polygons are provided within subfolders (poly*/poly*.shp)

In [ ]:
i=region_ID-1
subregion_name='zone'+format(i+1,'1d')
print("Start processing region :", subregion_name, "\n",
      "\n")

#Load Suberegion
ds_shp = gpd.read_file(shp_dir+'/poly'+format(i+1,'02d')+'/poly'+format(i+1,'02d')+'.shp')
shp_geo=ds_shp.geometry[ds_shp.NAME == subregion_name]
coords = ds_shp.geometry.exterior.get_coordinates()
coords

### Load Copernicus Marine Data
You may need to enter your credentials if environment variable were not set properly (cf. copernicus marine client documentation)

In [ ]:
%%time

#Get subregion bounding box (to nearest degree)
lon_range = np.floor(np.min(coords.x)),np.ceil(np.max(coords.x))
lat_range = np.floor(np.min(coords.y)),np.ceil(np.max(coords.y))

#Extract FCST data from MDS arco-time-chunked zarr datacube
ds_fcst = cmc.load_xarray_dataset(
    dataset_id = dataset_id_fcst[0],
    minimal_longitude = min(lon_range),
    maximal_longitude = max(lon_range),
    minimal_latitude = min(lat_range),
    maximal_latitude = max(lat_range),
    minimal_depth = 0.,
    maximal_depth = 0.,
    start_datetime = period_fcst[0],
    end_datetime = period_fcst[1],
    variables = variables,
    force_service = 'arco-time-series'
)

#Extract RAN data from MDS arco-geo-chunked zarr datacube
ds = cmc.load_xarray_dataset(
    dataset_id = dataset_id[0],
    minimal_longitude = min(lon_range),
    maximal_longitude = max(lon_range),
    minimal_latitude = min(lat_range),
    maximal_latitude = max(lat_range),
    minimal_depth = 0.,
    maximal_depth = 0.,
    start_datetime = period[0],
    end_datetime = period[1],
    variables = variables,
    force_service = 'arco-geo-series'
)

#### Display forecast dataset

In [ ]:
ds_fcst

#### Display RAN dataset

In [ ]:
ds

## Computational part

### Dataset tidying: Masking,Stacking, NaNs dropping
In order to optimize computation and data usage we will:
1. Mask RAN data over space using shapefiles (using the `regionmask.mask_geopandas` function)

***NOTE** This is one of the heaviest part as it loads ZARR chunks on-the-fly over the full requested subset of dataset*

In [ ]:
%%time
#Apply geospatial mask on RAN data (!SLOW)
mask_subregion = regionmask.mask_geopandas(shp_geo, ds.longitude, ds.latitude)
ds_sub = ds.where(mask_subregion==mask_subregion.max())


In [ ]:
%%time
#Apply geospatial mask on FSCT data (fast!)
mask_subregion_fcst = regionmask.mask_geopandas(shp_geo,ds_fcst.longitude, ds_fcst.latitude)
ds_fcst_sub = ds_fcst.where(mask_subregion==mask_subregion.max())

2. Subset RAN data over time to extract only `dayofyear` timeseries days that are contained within forecasting dates

In [ ]:
#Select fcst time steps for which compute climatological values
dates_sub = pd.DatetimeIndex(ds_sub.time)
dates_fcst = pd.DatetimeIndex(ds_fcst_sub.time)

#Get dayofyear value
A = dates_sub.dayofyear
B = dates_fcst.dayofyear

time_mask=np.full(len(A), False)
for i,b in enumerate(B): time_mask+=(A==b) #get interesection mak

ds_sub=ds_sub.isel(time=time_mask)
dates_sub=pd.DatetimeIndex(ds_sub.time)

ds_sub

3. As we want avergeing over shapefile area, we could stack the geospatial dimensions ...
4. ... and drop all masked values from furt

In [ ]:
%%time
#stack over space and drop NaNs
ds_stacked=ds_sub.stack(geo=("longitude","latitude")).dropna(dim="geo",how="any")
ds_stacked

### Compute climatological values
We will then extract from the clean RAN subsets:
* climatological averages over shapefile subregion for each day of year involved in the analysis
* same for the 90th percentile values

In [ ]:
%%time
#Compute 90th percentile
ds_clim_sub = ds_stacked.groupby("time.dayofyear").mean(dim="time")
ds_perc_sub = ds_stacked.groupby("time.dayofyear").quantile(0.9,dim="time")

#Average of subregion 
subregion_clim_avg=ds_clim_sub.thetao.mean(dim='geo')
subregion_perc_avg=ds_perc_sub.thetao.mean(dim='geo')

In [ ]:
ds_stacked.thetao

In [ ]:
fig=plt.figure()
plt.plot(dates_sub.dayofyear,ds_stacked.thetao.mean(dim="geo"),'.k',markersize=0.7)
plt.plot(dates_fcst.dayofyear, subregion_clim_avg,'+-r')
plt.plot(dates_fcst.dayofyear,subregion_perc_avg,'+-g')
plt.plot(dates_fcst.dayofyear,ds_fcst_sub.thetao.mean(dim=("latitude","longitude")),'--b')
fig.autofmt_xdate()
plt.xlabel('Day of year')
plt.ylabel(f'SST ({chr(176)}C)')
plt.title(f'Climatological mean vs 90th percentile over forecasting period :\n {period_fcst[0]} to {period_fcst[1]}')
plt.legend(['Raw GLORYS12 data','Climatology GLORYS12','90th Percentile GLORYS12 1993-2016','GLO12V4 Forecast'])

## Compute FCST anomalies
Anomalies will be computed against climatological mean and 90th percentile compute above

In [ ]:
%%time
#Compute FCST anomalies

#Extract first and last forecasting days
fcst_avg_d0=ds_fcst_sub.isel(time=0).mean()
fcst_avg_d9=ds_fcst_sub.isel(time=-1).mean()

#Remap
fcst_avg_map_d0=ds_fcst_sub.isel(time=0)*0
fcst_avg_map_d0+=fcst_avg_d0
fcst_avg_map_d9=ds_fcst_sub.isel(time=0)*0
fcst_avg_map_d9+=fcst_avg_d9

#Compute anomalies to climatology
fcst_anom_d0=ds_fcst_sub.isel(time=0)*0
fcst_anom_d0+=(fcst_avg_d0.thetao-subregion_clim_avg[0])

fcst_anom_d9=ds_fcst_sub.isel(time=-1)*0
fcst_anom_d9+=(fcst_avg_d9.thetao-subregion_clim_avg[-1])

#Compute difference to 90th percentile
fcst_perc_anom_d0=ds_fcst_sub.isel(time=0)*0
fcst_perc_anom_d0+=(fcst_avg_d0.thetao-subregion_perc_avg[0])

fcst_perc_anom_d9=ds_fcst_sub.isel(time=-1)*0
fcst_perc_anom_d9+=(fcst_avg_d9.thetao-subregion_perc_avg[-1])/(subregion_perc_avg[-1]-subregion_clim_avg[-1])

#Compute MHW severity
fcst_severity_d0=fcst_perc_anom_d0/(subregion_perc_avg[-1]-subregion_clim_avg[-1])
fcst_severity_d9=fcst_perc_anom_d9/(subregion_perc_avg[-1]-subregion_clim_avg[-1])

#### Show control metrics

In [ ]:
print(f'FCST mean (D0 / D+9): {fcst_avg_d0.thetao.values:0.1f}{chr(176)} - {fcst_avg_d9.thetao.values:0.1f}{chr(176)}')
print(f'FCST anom to RAN clim (D0 - D+9): {fcst_avg_d0.thetao.values-subregion_clim_avg.values[0]:0.1f}{chr(176)} - {fcst_avg_d9.thetao.values-subregion_clim_avg.values[-1]:0.1f}{chr(176)} (vs. {subregion_clim_avg.values[0]:0.1f}{chr(176)} - {subregion_clim_avg.values[-1]:0.1f}{chr(176)})')
print(f'FCST diff to RAN p90 (D0 - D+9): {fcst_avg_d0.thetao.values-subregion_perc_avg.values[0]:0.1f}{chr(176)} - {fcst_avg_d9.thetao.values-subregion_perc_avg.values[-1]:0.1f}{chr(176)} (vs. {subregion_perc_avg.values[0]:0.1f}{chr(176)} - {subregion_perc_avg.values[-1]:0.1f}{chr(176)})')

print('\n')

## Plot results

### Initialize plots

In [ ]:
#Overall plotting region
# Whole Iceland
# plt_lon = [-35,-3]
# plt_lat = [57,70]
# Subregion zoom
plt_lon = [lon_range[0]-np.floor((lon_range[1]-lon_range[0])*0.3),
           lon_range[1]+np.ceil((lon_range[1]-lon_range[0])*0.3)]
plt_lat = [lat_range[0]-np.floor((lat_range[1]-lat_range[0])*0.3),
           lat_range[1]+np.ceil((lat_range[1]-lat_range[0])*0.3)]

res = '50m'


#Initialize mapping
vars=[ds_fcst_sub.isel(time=0),fcst_anom_d0,fcst_severity_d0,
     ds_fcst_sub.isel(time=-1),fcst_anom_d9,fcst_severity_d9,]

titles=[f'FSCT SST avg D0',
        f'FSCT anom D0 (clim={subregion_clim_avg.values[0]:0.1f}{chr(176)})',
        f'FSCT MHW Intensity D0\n(-1=No MHW,0=Moderate,\n1=Strong,2=Severe,3=Extreme)',
        f'FSCT SST avg D+9',
        f'FSCT anom D+9 (clim={subregion_clim_avg.values[-1]:0.1f}{chr(176)})',
        f'FSCT MHW Intensity D9\n(-1=No MHW,0=Moderate,\n1=Strong,2=Severe,3=Extreme)']

cmaps=['Spectral_r','RdBu_r','YlOrBr',
       'Spectral_r','RdBu_r','YlOrBr']

sst_min=np.nanmin((fcst_avg_map_d0.thetao,fcst_avg_map_d9.thetao))
sst_max=np.nanmax((fcst_avg_map_d0.thetao,fcst_avg_map_d9.thetao))


levels=[(np.floor(sst_min-0.25),np.ceil(sst_max+0.25),0.25),
        (-3, 3.5, 0.5),
        (0., 4.5, 1.),
        (np.floor(sst_min-0.25),np.ceil(sst_max+0.25),0.25),
        (-3, 3.5, 0.5),
        (0., 4.5, 1.)]

#Initialized figure
fig= plt.figure(figsize=(10, 6))
# fig.subplots_adjust(hspace=0.2,wspace=0.05)
proj = ccrs.PlateCarree()

#Loop over subplots
i=0
axes=[]
subplt=[]
for col in range(3):
    for row in range(2):
        
        axes+= [plt.subplot(2, 3, i+1, projection=proj)]   
        axes[i].set_extent(plt_lon+plt_lat, crs=proj)

        levs=np.arange(levels[i][0],levels[i][1],levels[i][2])
        cmap=plt.get_cmap(cmaps[i],len(levs))
        
        subplt+=[vars[i].thetao.plot.pcolormesh(ax=axes[i],transform=proj, colors=cmap, levels=levs, add_colorbar=False)]
        shp_geo.plot(ax=axes[i], transform=proj, facecolor="None", edgecolor="black")        
        
        axes[i].set_title(titles[i])
        gl = axes[i].gridlines(
            draw_labels=True, linewidth=2, color='gray', alpha=0.5, linestyle='--'
        )
        
        axes[i].coastlines(resolution=res, color='black')
        axes[i].add_feature(cfeature.STATES, linewidth=0.3, edgecolor='brown')

        i+=1
fig.tight_layout()
fig.colorbar(subplt[3], orientation='horizontal', fraction=0.075, pad=0.05, ax=axes[3]);
fig.autofmt_xdate()
fig.colorbar(subplt[4], orientation='horizontal', fraction=0.075, pad=0.05, ax=axes[4]);
fig.colorbar(subplt[5], orientation='horizontal', fraction=0.075, pad=0.05, ax=axes[5]);

